In [22]:
import numpy as np

In [23]:
# 特征0：0：青年，1：中年，2：老年
# 特征1：0：无工作，1：有工作
# 特征2：0：无房产，1：有房产
# 特征3：0：信贷一般，1：信贷好，2：信贷非常好
X = np.array([[0,0,0,0],
              [0,0,0,1],
              [0,1,0,1],
              [0,1,1,0],
              [0,0,0,0],
              [1,0,0,0],
              [1,0,0,1],
              [1,1,1,1],
              [1,0,1,2],
              [1,0,1,2],
              [2,0,1,2],
              [2,0,1,1],
              [2,1,0,1],
              [2,1,0,2],
              [2,0,0,0]])
# 标签：0：不贷款，1：贷款
y = np.array([0,0,1,1,0,0,0,1,1,1,1,1,1,1,0])

In [24]:
def multi(y):
    ySet = set(y)
    bestCount = 0
    for yi in ySet:
        count = y.count(yi)
        if count > bestCount:
            bestCount = count
            bestyi = yi
    return bestyi

In [39]:
def gini(y):
    ySet = set(y)
    ret, n = 1, y.shape[0]
    for yi in ySet:
        ret -= (y[y==yi].shape[0]/n)**2
    return ret

def CART(X, y):
    # 若D中所有实例属于同一类$$C_k$$
    if len(set(y))==1:
        # 将类$$C_k$$作为该结点的类标记
        return {'label':y[0], 'gini':0}
    bestGini = np.inf
    # 对每个特征feature的每个取值value
    for feature in range(X.shape[1]):
        for value in set(X[:,feature]):
            # 将X分为$$R_1$$和$$R_2$$两个集合
            y1 = y[X[:,feature]<= value]
            y2 = y[X[:,feature]> value]
            # 计算$$R_1$$和$$R_2$$的基尼指数之和
            sumGini = gini(y1) + gini(y2)
            # 选择基尼指数计算结果最小的(feature, value)作为当前的最优划分
            if sumGini < bestGini:
                bestFeature, bestValue, bestGini = feature, value, sumGini
    # 基于最优划分生成2个子结点，将数据分配到两个子结点中
    node = {'feature':bestFeature,
            'value':bestValue,
            'gini':gini(y),
            'left':CART(X[X[:,bestFeature]<= bestValue], y[X[:,bestFeature]<= bestValue]),
           'right':CART(X[X[:,bestFeature]> bestValue], y[X[:,bestFeature]> bestValue])}
    return node
   

In [40]:
tree = CART(X, y)
tree

{'feature': 2,
 'value': 0,
 'gini': 0.48,
 'left': {'feature': 1,
  'value': 0,
  'gini': 0.4444444444444445,
  'left': {'label': 0, 'gini': 0},
  'right': {'label': 1, 'gini': 0}},
 'right': {'label': 1, 'gini': 0}}

In [45]:
def isLeaf(Node):
    # return type(Node).__name__ != 'dict'
    return not ('left' in Node.keys() or 'right' in Node.keys())
    
def calcAlphaList(Node, alphaList):
    if isLeaf(Node):
        return
    costNotSplit = Node['gini']
    costSplit = Node['left']['gini'] + Node['right']['gini']
    alphaList.append((costNotSplit-costSplit)/(Node['Tt']-1))
    calcAlphaList(Node['left'], alphaList)
    calcAlphaList(Node['right'], alphaList)
    
def calcTt(Node):
    if isLeaf(Node):
        return 1
    Node['Tt'] = calcTt(Node['left']) + calcTt(Node['right'])
    return Node['Tt']

def prune(Tree):
    calcTt(Tree)
    alphaList = []
    calcAlphaList(Tree, alphaList)
    print (alphaList)

In [46]:
prune(tree)

[0.017777777777777753, 0.4444444444444445]
